# Scientific Computing with NumPy and SciPy

----

By Adam A Miller (Northwestern/CIERA/SkAI)  
10 Sept 2025

## Learning Objectives
By the end of this notebook, you will be able to:

- Create and manipulate NumPy arrays (shapes, dtypes).
- Use slicing, boolean masking, and fancy indexing.
- Understand broadcasting and vectorized operations.
- Distinguish views vs. copies, and know when to use `.copy()` safely.
- Use selected SciPy submodules: `scipy.constants`, `scipy.integrate`, and `scipy.interpolate`.
- Apply these tools to small, realistic scientific tasks.

### Why Use NumPy for Scientific Computing?

- NumPy is the **foundation of numerical computing** in Python.


- Provides **efficient array operations** and **broadcasting**.


- Enables **vectorized computations** that are faster and more readable than pure Python loops.


Consider my previously declared statement that python is not great at `for` loops. `NumPy` performs basic array operations much faster than a for loop or list comprehension.

In [4]:
import numpy as np
import time

size = 1_000_000
a = list(range(size))
b = list(range(size))

# Pure Python loop
start = time.time()
c = np.empty_like(a)
for num, (a1, b1) in enumerate(zip(a, b)):
    c[num] = a1 + b1
end = time.time()
print(f"for loop time: {end - start:.4f} seconds")

# List comprehension
start = time.time()
c = [a[i] + b[i] for i in range(size)]
end = time.time()
print(f"list comprehension time: {end - start:.4f} seconds")

# NumPy vectorized operation
a_np = np.arange(size)
b_np = np.arange(size)

start = time.time()
c_np = a_np + b_np
end = time.time()
print(f"NumPy time: {end - start:.4f} seconds")


for loop time: 0.1781 seconds
list comprehension time: 0.0266 seconds
NumPy time: 0.0013 seconds


#### Vectorization

`numpy` enables vector operations (i.e., perform the same operation on an entire array "simultaneously").

Vectorization provides many improvements over the methods we have otherwise learned about this week: 

- It eliminates explicit loops


- It improves performance


- It makes code more concise and readable


In [5]:
# very quick example - suppose you wanted to plot a sine curve between 0 and 2*pi

x = np.linspace(0, 2*np.pi, 1000)
y = np.sin(x) # Done!

`numpy` can perform basic statistics on the data arrays:

In [6]:
arr = np.array([1, 2, 3, 4, 5])

# Basic stats
print(f"Mean: {arr.mean()}, Std: {arr.std()}, Sum: {arr.sum()}")


Mean: 3.0, Std: 1.4142135623730951, Sum: 15



### NumPy for Linear Algebra

(*Note* – can leverage duck typing to perform linear algebra on 2D arrays)

- Efficient matrix operations: multiplication, transpose, inverse

- Solving systems of equations

- Eigenvalue decomposition

In [7]:
# Define matrices
A = np.array([[1, 2], [3, 4]])
B = np.array([[2, 0], [1, 2]])

# Matrix multiplication
C = A @ B
print(f"Matrix product A @ B:\n{C}")

# Transpose
print(f"Transpose of A:\n{A.T}")

# Determinant
det_A = np.linalg.det(A)
print(f"Determinant of A: {det_A:.2f}")

# Inverse
inv_A = np.linalg.inv(A)
print(f"Inverse of A:\n{inv_A}")

# Eigenvalues and eigenvectors
eigvals, eigvecs = np.linalg.eig(A)
print(f"Eigenvalues: {eigvals}")
print(f"Eigenvectors:\n{eigvecs}")


Matrix product A @ B:
[[ 4  4]
 [10  8]]
Transpose of A:
[[1 3]
 [2 4]]
Determinant of A: -2.00
Inverse of A:
[[-2.   1. ]
 [ 1.5 -0.5]]
Eigenvalues: [-0.37228132  5.37228132]
Eigenvectors:
[[-0.82456484 -0.41597356]
 [ 0.56576746 -0.90937671]]


Quick note - when manipulating `numpy` arrays it is very important to know the different between a view and a copy. 


- *view*: Shares memory with the original array. Changes to the view affect the original.
- *copy*: Independent memory. Changes do **not** affect the original array.

- Views are **faster** and more memory-efficient.
- Copies are **safer** when you need to preserve original data.

Here's a weird thing: 

In [8]:
a = np.array([10, 20, 30, 40, 50])

# Create a view
view = a[1:4]
view[0] = 99

print(f"Original array after modifying view: {a}")
print(f"View: {view}")

# Create a copy
copy = a[1:4].copy()
copy[0] = 77

print(f"Original array after modifying copy: {a}")
print(f"Copy: {copy}")


Original array after modifying view: [10 99 30 40 50]
View: [99 30 40]
Original array after modifying copy: [10 99 30 40 50]
Copy: [77 30 40]


(just as I did earlier today, I'm going to populate the notebook with several additional useful examples but will not go over all the syntax during this lecture)

### Array Creation

In [10]:
# Creating arrays
a1 = np.array([1, 2, 3])
a2 = np.zeros((2, 3))
a3 = np.ones((3, 3), dtype=float)
a4 = np.arange(0, 10, 2)
a5 = np.linspace(0, 1, 5)
a6 = np.random.default_rng(42).normal(size=(3,4))

print(f'a1: {a1}')
print(f'a2: {a2}')
print(f'a3: {a3}')
print(f'a4: {a4}')
print(f'a5: {a5}')
print(f'a6 shape: {a6.shape}, dtype: {a6.dtype}')

a1: [1 2 3]
a2: [[0. 0. 0.]
 [0. 0. 0.]]
a3: [[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]
a4: [0 2 4 6 8]
a5: [0.   0.25 0.5  0.75 1.  ]
a6 shape: (3, 4), dtype: float64


### Slicing and Indexing

In [12]:
A = np.arange(1, 10).reshape(3, 3)
print('A =', A)

# Basic slicing
print('First row:', A[0])
print('Last column:', A[:, -1])
print('Center 2x2 block:', A[0:2, 1:3])

# Boolean masking
mask = A % 2 == 0
print('Even mask:', mask)
print('Even elements:', A[mask])

# Note: slices create views (no copy), fancy/boolean indexing create copies
B = A[:, :2]  # view
B[0, 0] = -99
print('After modifying B, A becomes:', A)

A = [[1 2 3]
 [4 5 6]
 [7 8 9]]
First row: [1 2 3]
Last column: [3 6 9]
Center 2x2 block: [[2 3]
 [5 6]]
Even mask: [[False  True False]
 [ True False  True]
 [False  True False]]
Even elements: [2 4 6 8]
After modifying B, A becomes: [[-99   2   3]
 [  4   5   6]
 [  7   8   9]]


### Broadcasting and Basic Operations

In [13]:
x = np.array([1., 2., 3.])
y = np.array([[10.], [20.], [30.]])
print('x + y =', x + y)
print('2*x =', 2 * x)
print('x**2 =', x**2)
print('Sum over rows of (x+y):', (x+y).sum(axis=1))

# Broadcasting pitfalls: shape alignment
try:
    _ = np.ones((3,1)) + np.ones((2,))
except ValueError as e:
    print('Broadcasting error example:', e)

x + y = [[11. 12. 13.]
 [21. 22. 23.]
 [31. 32. 33.]]
2*x = [2. 4. 6.]
x**2 = [1. 4. 9.]
Sum over rows of (x+y): [36. 66. 96.]


### Why Use SciPy for Scientific Computing?

(especially when we already have NumPy...)

[`scipy`](https://scipy.org/) builds on NumPy to provide advanced scientific computing tools. This includes specialized (and faster) modules for: 




  - Integration


  - Interpolation


  - Optimization (Friday!)


  - Signal processing


  - Physical constants


If you want to model, simulate, or analyze scientific data in python then `scipy` is for you!

#### Physical constants

- Provides **precise values** for hundreds of physical constants.
- Includes **unit conversions** for pressure, energy, temperature, etc.


In [14]:
from scipy import constants

# Access physical constants
print(f"Speed of light: {constants.c} m/s")
print(f"Planck constant: {constants.h} J·s")
print(f"Gravitational constant: {constants.G:.3e} m^3·kg^-1·s^-2") # update the Orrery!!

# Convert units
print(f"1 electronvolt in joules: {constants.eV} J")
print(f"1 atmosphere in pascals: {constants.atm} Pa")


Speed of light: 299792458.0 m/s
Planck constant: 6.62607015e-34 J·s
Gravitational constant: 6.674e-11 m^3·kg^-1·s^-2
1 electronvolt in joules: 1.602176634e-19 J
1 atmosphere in pascals: 101325.0 Pa


### Interpolation

`scipy.interpolate` provides multiple useful methods for interpolation: 


- `interp1d`: 1D interpolation (linear, cubic, etc.)
- `griddata`: Interpolation on scattered data
- `UnivariateSpline`: Spline fitting with smoothing


In [15]:
from scipy import interpolate

# Sample data
x = np.linspace(0, 10, 10)
y = np.sin(x)

# Create interpolator
f_interp = interpolate.interp1d(x, y, kind='cubic')

# Evaluate at new points
x_new = np.linspace(0, 10, 100)
y_new = f_interp(x_new)

print(f"Interpolated values at x=6.283: {f_interp(6.283):.4f}")


Interpolated values at x=6.283: 0.0006


#### Integration

`scipy.integration` enables basic (numeric) calculus:

- `quad`: Integrate a function over a range
- `dblquad`, `tplquad`: Double and triple integrals
- `solve_ivp`: Solve initial value problems for OD


In [16]:
from scipy import integrate

# Define a function to integrate
def f(x):
    return np.sin(x)

result, error = integrate.quad(f, 0, np.pi) # Integrate from 0 to pi
print(f"Integral of sin(x) from 0 to pi: {result:.4f}, error estimate: {error:.2e}")


Integral of sin(x) from 0 to pi: 2.0000, error estimate: 2.22e-14
